In [1]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import LabelBinarizer
from sklearn import preprocessing
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import mean_squared_error
import matplotlib.pyplot as plt
from sklearn.model_selection import GroupKFold
from sklearn import mixture
from tqdm import tqdm
import warnings
warnings.simplefilter('ignore')

df = pd.read_csv("~/Desktop/haru_5_7/data/train.csv") 
df_test = pd.read_csv("~/Desktop/haru_5_7/data/test.csv") 



In [271]:
# GMM
def gmm(train_df,test_df):
    clf = mixture.GaussianMixture(n_components=10, covariance_type='full')
    gm = clf.fit_predict(pd.concat([train_df, test_df])[['lat','lon']])

    train_df['gmm'] = gm[:len(train_df)]
    test_df['gmm'] = gm[len(train_df):]
    return train_df,test_df



## 重心からの距離
def kyori(train_df,test_df):
    all_df = pd.concat([train_df, test_df])
    lat = all_df['lat'].mean()
    long = all_df['lon'].mean()

    centroid = (lat, long)

    centroid_dist = []
    for idx, row in tqdm(train_df.iterrows(), total=len(train_df)):
        a = np.array([row['lat'], row['lon']])
        centroid_dist.append(np.linalg.norm(a - centroid))

    train_df['centroid_dist'] = centroid_dist

    centroid_dist = []
    for idx, row in tqdm(test_df.iterrows(), total=len(test_df)):
        a = np.array([row['lat'], row['lon']])
        centroid_dist.append(np.linalg.norm(a - centroid))

    test_df['centroid_dist'] = centroid_dist
    
    return train_df,test_df


def label(train_df,test_df):
    categoricals = ['Country', "City",'gmm']
    for c in categoricals:
        le = LabelEncoder()
        le.fit(train_df.append(test_df)[c].astype(str))
        train_df[c] = le.transform(train_df[c].astype(str))
        test_df[c] = le.transform(test_df[c].astype(str))
    return train_df,test_df



def count(train_df,test_df):
    count_enc = ['Country',"City" ,'gmm']
    for c in count_enc:
        _count = train_df.append(test_df)[c].astype(str).value_counts().to_dict()
        train_df[f'{c}_count'] = train_df[c].astype(str).map(_count)
        test_df[f'{c}_count'] = test_df[c].astype(str).map(_count)
    return train_df,test_df



df,df_test = gmm(df,df_test)
df,df_test = kyori(df,df_test)
df,df_test = label(df,df_test)
df,df_test = count(df,df_test)

df

100%|██████████| 53509/53509 [00:02<00:00, 20527.14it/s]


,id,year,month,day,Country,City,lat,lon,co_cnt,co_min,...,dew_min,dew_mid,dew_max,dew_var,pm25_mid,gmm,centroid_dist,Country_count,City_count,gmm_count
0,1,2019,1,1,0,24,-27.46794,153.02809,38,0.749,...,7.671,10.358,15.112,13.424,19.901,6,107.139995,7496,1009,7496
1,2,2019,1,1,0,54,-12.46113,130.84185,47,2.594,...,21.324,23.813,24.221,2.021,13.741,6,80.367204,7496,1057,7496
2,3,2019,1,1,0,146,-37.81400,144.96332,17,1.190,...,10.309,13.133,15.422,6.355,25.918,6,106.717474,7496,1038,7496
3,4,2019,1,1,0,176,-32.92953,151.78010,63,4.586,...,7.146,10.685,13.344,9.417,174.370,6,109.156753,7496,717,7496
4,5,2019,1,1,0,193,-31.95224,115.86140,47,4.689,...,1.091,3.277,12.272,4.109,167.063,6,81.773781,7496,964,7496
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
195936,195937,2021,12,31,28,103,30.33218,-81.65565,12,0.694,...,16.774,22.679,26.058,13.252,16.150,4,144.402554,11133,779,13438
195937,195938,2021,12,31,28,133,36.17497,-115.13722,14,0.528,...,10.432,14.741,15.827,7.078,16.895,1,177.983621,11133,953,7867
195938,195939,2021,12,31,28,150,43.03890,-87.90647,171,1.975,...,2.049,3.531,6.686,5.286,86.299,4,151.197442,11133,1040,13438
195939,195940,2021,12,31,29,88,21.02450,105.84117,31,2.613,...,1.922,7.443,7.716,4.642,36.523,7,44.065146,2570,942,52652


In [259]:
# 目的変数と説明変数の定義
X = df.drop(columns=['pm25_mid'])
y = df['pm25_mid']

df_test = df_test.drop(columns=["id"])

print("finish")

finish


In [272]:
import numpy as np
import pandas as pd

# グラフ描画用
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline
sns.set_style('darkgrid')

import lightgbm as lgb

from sklearn.datasets import make_classification
from sklearn.metrics import roc_auc_score, roc_curve, accuracy_score
from sklearn.model_selection import train_test_split
from sklearn.model_selection import GroupKFold
from sklearn.model_selection import KFold

In [273]:
# 学習用のパラメータ
params = {
    'objective': 'regression',    # 回帰を指定
    'metric': 'rmse',             # 回帰の評価関数
    'learning_rate': 0.1,         # 学習率
    "boosting" : "gbdt",        # default
    "seed": 42
}


FOLD = 10
#FOLD = 5
NUM_ROUND = 100
VERBOSE_EVAL = -1



# TargetEncording col
target_enc = ['Country', 'gmm',"yy_mm_dd_country"]
target_enc_key = ['pm25_mid']

#group = X['City']
#group = X['yaermonth_ci']
valid_scores = []
valid_pred = []
valid_ans = []
models = []
ids = []
#kf = GroupKFold(n_splits=FOLD)
kf = KFold(n_splits=FOLD, shuffle=True, random_state=0)


for fold, (train_indices, valid_indices) in enumerate(kf.split(X)):
#for fold, (train_indices, valid_indices) in enumerate(kf.split(X, y, group)):
    X_train, X_valid = X.iloc[train_indices], X.iloc[valid_indices]
    y_train, y_valid = y.iloc[train_indices], y.iloc[valid_indices]
    
    ids.append(X_valid["id"])
    X_train = X_train.drop(columns=["id"])
    X_valid = X_valid.drop(columns=["id"])
    """
    # Target Encoding
    if len(target_enc) > 0:
        for t in target_enc_key:
            for c in target_enc:
                X_train[f'{c}_target_enc_by_{t}'] = df.iloc[train_indices][c].map(df.iloc[valid_indices].groupby(c)[t].mean().to_dict())
                X_valid[f'{c}_target_enc_by_{t}'] = df.iloc[valid_indices][c].map(df.groupby(c)[t].mean().to_dict())
                df_test[f'{c}_target_enc_by_{t}'] = df_test[c].map(df.groupby(c)[t].mean().to_dict())
    """
    lgb_train = lgb.Dataset(X_train, y_train)
    lgb_eval = lgb.Dataset(X_valid, y_valid)
    
    
    # model の作成、学習
    model = lgb.train(params=params,
                      train_set=lgb_train,
                      valid_sets=[lgb_train, lgb_eval],
                      num_boost_round=10000,           # 10000 round まで実行
                      early_stopping_rounds=100,        # 100 round ごとに los を確認。改善なければ stop
                      verbose_eval=100                  # 100 round ごとの los を表示。
                     )
    
    # validation データでの予測結果
    val_pred = model.predict(X_valid)
    print(np.exp(val_pred))
    score = np.sqrt(mean_squared_error(y_valid,val_pred))
    valid_scores.append(score)
    valid_pred.append(val_pred)
    valid_ans.append(y_valid)
    
    models.append(model)
    print()
    print(f'fold {fold} RMSE: {score}')
    print()
    
cv_score = np.mean(valid_scores)
print()
print("==="*20)
print(f'CV score: {cv_score}')
print()
print("best socre : 19.25994515547478")

[LightGBM] [Warning] Auto-choosing col-wise multi-threading, the overhead of testing was 0.013908 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 12728
[LightGBM] [Info] Number of data points in the train set: 176346, number of used features: 57
[LightGBM] [Info] Start training from score 59.160839
Training until validation scores don't improve for 100 rounds
[100]	training's rmse: 19.8994	valid_1's rmse: 20.6966
[200]	training's rmse: 18.9217	valid_1's rmse: 20.1799
[300]	training's rmse: 18.2677	valid_1's rmse: 19.9625
[400]	training's rmse: 17.7396	valid_1's rmse: 19.8472
[500]	training's rmse: 17.2726	valid_1's rmse: 19.7581
[600]	training's rmse: 16.8642	valid_1's rmse: 19.6966
[700]	training's rmse: 16.473	valid_1's rmse: 19.6556
[800]	training's rmse: 16.0979	valid_1's rmse: 19.6077
[900]	training's rmse: 15.7747	valid_1's rmse: 19.5814
[1000]	training's rmse: 15.4536	valid_1's rmse: 19.567
[1100]	training's rmse: 15.1554	valid_1's

[4.22064572e+62 5.58065082e+28 9.98768140e+53 ... 1.15274365e+21
 3.86372861e+27 2.35490938e+24]

fold 4 RMSE: 19.517308775986344

[LightGBM] [Warning] Auto-choosing col-wise multi-threading, the overhead of testing was 0.013030 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 12731
[LightGBM] [Info] Number of data points in the train set: 176347, number of used features: 57
[LightGBM] [Info] Start training from score 59.206586
Training until validation scores don't improve for 100 rounds
[100]	training's rmse: 19.9301	valid_1's rmse: 20.4296
[200]	training's rmse: 18.9316	valid_1's rmse: 19.9757
[300]	training's rmse: 18.2798	valid_1's rmse: 19.7771
[400]	training's rmse: 17.7667	valid_1's rmse: 19.6627
[500]	training's rmse: 17.2972	valid_1's rmse: 19.5914
[600]	training's rmse: 16.8757	valid_1's rmse: 19.5269
[700]	training's rmse: 16.5048	valid_1's rmse: 19.486
[800]	training's rmse: 16.1426	valid_1's rmse: 19.4402
[900]	training's rms

[900]	training's rmse: 15.8411	valid_1's rmse: 19.4818
[1000]	training's rmse: 15.5204	valid_1's rmse: 19.4437
[1100]	training's rmse: 15.2165	valid_1's rmse: 19.4159
[1200]	training's rmse: 14.948	valid_1's rmse: 19.3916
[1300]	training's rmse: 14.6769	valid_1's rmse: 19.3776
[1400]	training's rmse: 14.4133	valid_1's rmse: 19.3604
[1500]	training's rmse: 14.156	valid_1's rmse: 19.3516
[1600]	training's rmse: 13.9102	valid_1's rmse: 19.3348
[1700]	training's rmse: 13.6733	valid_1's rmse: 19.328
[1800]	training's rmse: 13.4537	valid_1's rmse: 19.3218
[1900]	training's rmse: 13.2344	valid_1's rmse: 19.3119
[2000]	training's rmse: 13.0248	valid_1's rmse: 19.3088
[2100]	training's rmse: 12.819	valid_1's rmse: 19.3007
[2200]	training's rmse: 12.6232	valid_1's rmse: 19.2964
[2300]	training's rmse: 12.428	valid_1's rmse: 19.2892
[2400]	training's rmse: 12.2266	valid_1's rmse: 19.2834
[2500]	training's rmse: 12.0405	valid_1's rmse: 19.2745
[2600]	training's rmse: 11.8504	valid_1's rmse: 19.268

In [274]:
# feature importanceを表示
X_col = X.drop(columns=["id"])
importance = pd.DataFrame(model.feature_importance(), index=X_col.columns, columns=['importance'])
display(importance["importance"].sort_values(ascending=False).head(20))

co_mid          2591
no2_min         2584
no2_mid         2477
co_min          2112
o3_mid          2110
so2_mid         2010
so2_min         1843
ws_mid          1722
o3_min          1707
humidity_mid    1686
dew_var         1636
ws_min          1616
o3_max          1607
lat             1605
so2_cnt         1588
so2_max         1566
pressure_var    1551
co_max          1546
month           1525
o3_var          1524
Name: importance, dtype: int32

In [262]:
sample = pd.read_csv("submit_sample.csv",header=None) 

pred = np.zeros(len(df_test))
for model in models:
    p_i = model.predict(df_test.values)
    pred += p_i
    
pred = pred / FOLD
pred = pd.Series(pred)
pred_df = pd.DataFrame(pred,columns=['pred'])

sample[1] = pred_df["pred"]
sample.to_csv("LGBT_single.csv",index=None,header=None)
sample

,0,1
0,195942,24.720375
1,195943,37.026119
2,195944,27.208136
3,195945,65.198687
4,195946,129.724993
...,...,...
53504,249446,76.731578
53505,249447,108.726955
53506,249448,72.382909
53507,249449,36.543225


In [197]:
for i in range(FOLD):
    if i == 0:
        pred_df = pd.DataFrame({ 'id' :ids[i],'pred':valid_pred[i]})
    else:
        pred_data = pd.DataFrame({ 'id' :ids[i],'pred':valid_pred[i]})
        pred_df = pd.concat([pred_df, pred_data]) 
        
pred_df = pred_df.sort_values("id").reset_index(drop=True)

pred_df["True"] = df["pm25_mid"]
print(np.sqrt(mean_squared_error(pred_df["True"], pred_df["pred"])))

# 目的変数と説明変数の定義
X["Psuedo"] = pred_df["pred"]

pred = np.zeros(len(df_test))
for model in models:
    p_i = model.predict(df_test.values)
    pred += p_i
    
pred = pred / FOLD
pred = pd.Series(pred)
pred_df = pd.DataFrame(pred,columns=['pred'])

df_test["Psuedo"] = pred_df["pred"]


print("Psuedo Labeling")

19.249558725789672
Psuedo Labeling


In [198]:
# 学習用のパラメータ
params = {
    'objective': 'regression',    # 回帰を指定
    'metric': 'rmse',             # 回帰の評価関数
    'learning_rate': 0.1,         # 学習率
    "boosting" : "gbdt",        # default
    "seed": 42
}


FOLD = 10
#FOLD = 5
NUM_ROUND = 100
VERBOSE_EVAL = -1


#group = X['City']
#group = X['yaermonth_ci']
valid_scores = []
valid_pred = []
valid_ans = []
models = []
ids = []
#kf = GroupKFold(n_splits=FOLD)
kf = KFold(n_splits=FOLD, shuffle=True, random_state=0)


for fold, (train_indices, valid_indices) in enumerate(kf.split(X)):
#for fold, (train_indices, valid_indices) in enumerate(kf.split(X, y, group)):
    X_train, X_valid = X.iloc[train_indices], X.iloc[valid_indices]
    y_train, y_valid = y.iloc[train_indices], y.iloc[valid_indices]
    
    ids.append(X_valid["id"])
    X_train = X_train.drop(columns=["id"])
    X_valid = X_valid.drop(columns=["id"])
    
    lgb_train = lgb.Dataset(X_train, y_train)
    lgb_eval = lgb.Dataset(X_valid, y_valid)
    
    
    # model の作成、学習
    model = lgb.train(params=params,
                      train_set=lgb_train,
                      valid_sets=[lgb_train, lgb_eval],
                      num_boost_round=10000,           # 10000 round まで実行
                      early_stopping_rounds=100,        # 100 round ごとに los を確認。改善なければ stop
                      verbose_eval=100                  # 100 round ごとの los を表示。
                     )
    
    # validation データでの予測結果
    val_pred = model.predict(X_valid)
    score = np.sqrt(mean_squared_error(y_valid, val_pred))
    valid_scores.append(score)
    valid_pred.append(val_pred)
    valid_ans.append(y_valid)
    
    models.append(model)
    print()
    print(f'fold {fold} RMSE: {score}')
    print()
    
cv_score = np.mean(valid_scores)
print()
print("==="*20)
print(f'CV score: {cv_score}')
print()

print("best socre : 19.25994515547478")

[LightGBM] [Warning] Auto-choosing col-wise multi-threading, the overhead of testing was 0.013272 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 12983
[LightGBM] [Info] Number of data points in the train set: 176346, number of used features: 58
[LightGBM] [Info] Start training from score 59.160839
Training until validation scores don't improve for 100 rounds
[100]	training's rmse: 18.6268	valid_1's rmse: 19.5128
Early stopping, best iteration is:
[63]	training's rmse: 18.8206	valid_1's rmse: 19.4926

fold 0 RMSE: 19.492605425863687

[LightGBM] [Warning] Auto-choosing col-wise multi-threading, the overhead of testing was 0.013346 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 12986
[LightGBM] [Info] Number of data points in the train set: 176347, number of used features: 58
[LightGBM] [Info] Start training from score 59.203123
Training until validation scores don't improve for 100 rounds
[1

In [239]:
df = pd.read_csv("~/Desktop/signate-main/data/train.csv") 

df["Psuedo"] = X["Psuedo"]
df["dif"] = abs(df["pm25_mid"]- df["Psuedo"])

df["Psuedo"] = df["Psuedo"].apply(lambda x : 200 if x > 300 else x)

from sklearn.preprocessing import StandardScaler

num_cols = ["ws_mid"]
scaler = StandardScaler()
scaler.fit(df[num_cols])
df[num_cols] = scaler.transform(df[num_cols])

df = df[df["dif"] > 20]
#df = df[df["dif"] < 30]
display(df[["pm25_mid","Psuedo","dif","co_mid","no2_mid","ws_mid"]])

print(np.sqrt(mean_squared_error(df["pm25_mid"],df["Psuedo"])))

,pm25_mid,Psuedo,dif,co_mid,no2_mid,ws_mid
3,174.370,145.430541,28.939459,11.044,17.471,-1.244950
4,167.063,139.128494,27.934506,8.681,21.140,-1.076834
5,110.224,142.642149,32.418149,7.159,16.734,-1.248286
10,61.635,108.598059,46.963059,0.124,18.863,-0.197559
12,77.193,33.275500,43.917500,2.572,2.595,1.164717
...,...,...,...,...,...,...
195926,61.016,145.888864,84.872864,8.356,18.464,-0.930733
195931,53.191,88.902725,35.711725,8.430,18.531,-0.901379
195934,133.977,62.439230,71.537770,6.055,8.252,0.073962
195936,16.150,36.234160,20.084160,0.995,1.071,0.227401


36.88953642634359


In [200]:
sample = pd.read_csv("submit_sample.csv",header=None) 

pred = np.zeros(len(df_test))
for model in models:
    p_i = model.predict(df_test.values)
    pred += p_i
    
pred = pred / FOLD
pred = pd.Series(pred)
pred_df = pd.DataFrame(pred,columns=['pred'])

sample[1] = pred_df["pred"]
sample.to_csv("LGBT_pseudo.csv",index=None,header=None)
sample

,0,1
0,195942,20.245027
1,195943,35.475699
2,195944,28.122477
3,195945,67.013706
4,195946,130.774131
...,...,...
53504,249446,72.623906
53505,249447,103.614481
53506,249448,70.626819
53507,249449,33.701942
